# Reproducing a Regression Table (Heteroskedastic Variance)
Suppose we have the following table of data.
```
     +--------------+
     |  y   x1   x2 |
     |--------------|
  1. |  3    4    5 |
  2. |  2    1    3 |
  3. |  9   11   18 |
  4. |  0    4   -2 |
  5. |  9    8    3 |
     |--------------|
  6. | 12    9   25 |
  7. |  3    7   18 |
  8. | 15   15   12 |
  9. |  4   16    8 |
 10. | 11   14   13 |
     +--------------+
```
Suppose we use the built-in command for regression in R: `lm`. Note that `type="HC"` is for heteroskedasticity robust standard errors, `type="HC1"` is for heteroskedasticty robust standard errors with bias correction.

In [1]:
library(lmtest)
library(sandwich)

df = data.frame(
    cbind(
    c(3, 2 ,9, 0, 9, 12, 3, 15, 4, 11),
    c(4, 1, 11, 4, 8, 9, 7, 15, 16, 14),
    c(5, 3, 18, -2, 3, 25, 18, 12, 8, 13))
)
N = dim(df)[1] # sample size
K = dim(df)[2] - 1 # number of regressors

colnames(df) <- c("y", "x1", "x2")
fit = lm(y ~ x1 + x2, data=df)
coeftest(fit, vcov=vcovHC(fit, type="HC"))

Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric




t test of coefficients:

            Estimate Std. Error t value Pr(>|t|)
(Intercept)  0.10417    1.43119  0.0728   0.9440
x1           0.50192    0.29387  1.7080   0.1314
x2           0.21638    0.14088  1.5359   0.1684


Can we reproduce the important numbers in the table? First let's start with the regression coefficients.

In [2]:
data = data.matrix(df)
Y = data[,1]
X = cbind(rep(1, ), data[, 2:3])

reg = function(Y, X){solve(t(X)%*%X)%*%t(X)%*%Y}
res = function(Y, X){Y - X%*%reg(Y, X)}

b.hat = reg(Y, X)
b.hat

,0.1041661
x1,0.5019225
x2,0.2163809


Next we will reproduce the (robust) standard error. 

- Recall from previous lessons that
$$
\sqrt{N}(\hat{\beta}_N - \beta) \overset{d}{\to} \mathcal{N}(0, V)
$$
where $V = E[XX']^{-1}Var[XU]E[XX']^{-1}$ so that the sample counterpart to $V$ is
$$
\hat{V}_N  = \left(\frac{1}{N}\boldsymbol{X}'\boldsymbol{X}\right)^{-1}\frac{1}{N}\left(\boldsymbol{X}'\hat{\Omega}\boldsymbol{X}\right)\left(\frac{1}{N}\boldsymbol{X}'\boldsymbol{X}\right)^{-1} 
$$
$$ = N\left(\boldsymbol{X}'\boldsymbol{X}\right)^{-1}\left(\boldsymbol{X}'\hat{\Omega}\boldsymbol{X}\right)\left(\boldsymbol{X}'\boldsymbol{X}\right)^{-1}
$$
where $\hat{\Omega}_N$ is an $N\times N$ diagonal matrix
$$
\hat{\Omega} =
\begin{pmatrix}
\hat{U}_1^2 & 0 & \cdots & \cdots & 0 \\
0 & \hat{U}_2^2 & 0 & \cdots & 0 \\
\vdots & 0 & \ddots & \vdots & \vdots \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
0 & \cdots & \cdots & 0 & \hat{U}_N^2 
\end{pmatrix}
$$
Thus, estimated variance of $\hat{\beta}_N$ is $\frac{\hat{V}_N}{N}$.
- Note: Stata calculates an unbiased version of the variance that corrects for degrees of freedom by estimating $Var[X_iU_i]$ with $\frac{1}{N-(K+1)}\boldsymbol{X}'\hat{\Omega}\boldsymbol{X}$ instead of $\frac{1}{N}\boldsymbol{X}'\hat{\Omega}\boldsymbol{X}$, where $K$ is the number of regressors (excluding the constant). We will not do this bias correction below so that we match the standard errors of R's output using the type="HC"

In [3]:
U.hat = res(Y, X)

# Multiplying row in X by corresponding element in U.hat
XU = sweep(X, MARGIN=1, U.hat, `*`)

# Calculating variance matrix
V = N*solve(t(X)%*%X)%*%t(XU)%*%XU%*%solve(t(X)%*%X)
# V = (N^2/(N-K))*solve(t(X)%*%X)%*%t(XU)%*%XU%*%solve(t(X)%*%X)
se = sqrt(diag(V)/N)
se

x1        x2 
1.4311874 0.2938699 0.1408795

Now the t-statistics.

In [4]:
t.stats = b.hat/se
t.stats

,0.07278296
x1,1.70797541
x2,1.53592934


And the p-values. By default, R uses a t-distribution (instead of a normal distribution, as we derived in class) with $N-K-1$ degrees of freedom. The answers won't change much if you use a normal cdf instead.

In [5]:
2*(1 - pt(t.stats, df=N-K-1))

,0.9440149
x1,0.1313962
x2,0.1684344


You can also perform a chi-squared test or F-test.

In [6]:
R = rbind(c(0, 1, 0), c(0, 0, 1))
q = dim(R)[1]
f.stat = N*t(R%*%b.hat)%*%solve(R%*%V%*%t(R))%*%(R%*%b.hat)/q
p.val = 1 - pf(f.stat, df1=q, df2=N-K-1)
c(f.stat, p.val)

[1] 11.486543670  0.006155778

# Reproducing a Regression Table (Homoskedastic Variance)
The default behavior in R's `lm` function is to perform statistics of OLS using homoskedastic variance. These errors assume that $E[U^2|X] = E[U^2]$. This will imply that
$$
V = E[XX']^{-1}Var[XU]E[XX']^{-1} = E[XX']^{-1}E[XX']E[U^2]E[XX']^{-1} = E[U^2]E[XX']^{-1}
$$
and therefore that
$$
\hat{V}_N = (\boldsymbol{X}'\boldsymbol{X})^{-1}\sum_{i=1}^N\hat{U}_i^2.
$$
We will use this new estimate of the variance matix to compute what R produces without manually calculating the robust version. The default behavior in R is to also use the bias corrected version of the homoskedastic variance, which we will also reproduce below.

In [7]:
summary(fit)


Call:
lm(formula = y ~ x1 + x2, data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-5.8660 -1.3894  0.2755  1.7407  4.7704 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)
(Intercept)   0.1042     2.6515   0.039    0.970
x1            0.5019     0.2783   1.803    0.114
x2            0.2164     0.1684   1.285    0.240

Residual standard error: 3.856 on 7 degrees of freedom
Multiple R-squared:  0.5426,	Adjusted R-squared:  0.4119 
F-statistic: 4.152 on 2 and 7 DF,  p-value: 0.06473


In [8]:
V.hom = (N/(N-K-1))*solve(t(X)%*%X)*sum(U.hat^2)
se = sqrt(diag(V.hom)/N)
se

x1        x2 
2.6515302 0.2783191 0.1684089

t-statistics and p-values are calculated in a similar way.

In [9]:
t.stats = b.hat/se
t.stats

,0.03928526
x1,1.80340648
x2,1.28485409


In [10]:
p.vals = 2*(1 - pt(t.stats, df=N-K-1))
p.vals

,0.9697599
x1,0.1143131
x2,0.2397226


In [11]:
f.stat = N*t(R%*%b.hat)%*%solve(R%*%V.hom%*%t(R))%*%(R%*%b.hat)/q
p.val = 1 - pf(f.stat, df1=q, df2=N-K-1)
c(f.stat, p.val)

[1] 4.15172144 0.06472635

You can also calculate the R-squared.

In [12]:
R2 = 1 - sum(U.hat^2)/sum((Y - mean(Y))^2)
R2

[1] 0.5425866

And using the $R^2$, in the homoskedastic case you can also get the F-statistic.

In [13]:
f.stat = ((R2)/(1-R2))*((N-K-1)/q)
f.stat

[1] 4.151721